<a href="https://colab.research.google.com/github/lightuse/AML/blob/master/Automated_Machine_Learning_Regression_with_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# supervised learning
## Regression

In [ ]:
!pip install optuna

In [ ]:
!pip install catboost

Setting

In [ ]:
from common import function

In [ ]:
pipelines = function.setup_algorithm(pipelines)

In [ ]:
exception_algorithm_list = ['tree', 'gbr1', 'gbr2', 'ols', 'ridge', 'knn', 'xgboost', 'catboost', 'rf']
for algorithm in exception_algorithm_list:
    if algorithm in pipelines:
        pipelines.pop(algorithm)
# Feature selection by RandomForestClassifier
feature_selection_rf_list = ['knn']
tuning_prarameter_list = ['lightgbm', 'rf', 'tree']
tuning_prarameter_list = []
options_evaluation = ['R2', 'MAE:neg_mean_absolute_error',
                      'RMSE:neg_root_mean_squared_error',
                      'MAPE:Mean Absolute Persentage Error',
                      'RMSLE:Root Mean Squared Logarithmic Error',
                      'RMSPE:Root Mean Squared Persentage Error']
options_evaluation = ['RMSE', 'MAE', 'R2', 'RMSLE']
# カテゴリ変数をリストで設定
ohe_columns = ['bed_type', 'city', 'cleaning_fee', 'host_identity_verified', 'property_type', 'room_type', 'neighbourhood']
# カテゴリ変数をobject型で読み込むための準備
my_dtype = {'thumbnail_url':object, 'amenities':object}
id_label = 'id'
target_label = 'y'
drop_columns = ['amenities', 'description', 'name', 'thumbnail_url', 'host_since', 'first_review', 'last_review', 'cancellation_policy', 'host_has_profile_pic']
out_put_data_dir = "/content/drive/My Drive/Colab Notebooks/AIQuest/data/"
train_file_name = out_put_data_dir + 'train.csv'
test_file_name = out_put_data_dir + 'test.csv'
model_columns_file_name = out_put_data_dir + 'model_columns.csv'
# feature_selection range 50-100
n_features_to_select = 100
# ファイル出力拡張子
file_extention = 'csv'
# ホールドアウト有無
is_holdout = False
# k-fold 法を利用するか
is_k_fold = True
# optuna を利用するかどうか 
is_optuna = True
# 目的変数をマイナスから0にするかどうか
is_target_minus_to_zero = True

In [ ]:
import pandas as pd
import numpy as np
def input_train_file(filename, my_dtype, id_label):
    df = pd.read_csv(train_file_name, header=0, dtype=my_dtype)
    #df = df[df['room_type'] == 'Shared room']
    #df = df[df['room_type'] == 'Entire home/apt']
    #df = df[df['room_type'] == 'Private room']
    df['y'] = np.log1p(df['y'])
    df = df[(df['y'] >= 1) & (df['y'] <= 8)]
    df = df.reset_index(drop=True)
    # データの形式に合わせて適時修正
    X  = df.iloc[:,:-1]
    X = X.drop(id_label, axis=1)
    X = X.reset_index(drop=True)
    ID = X.iloc[:,[0]]
    y  = df.iloc[:,-1]
    print('欠損個数（数値変数の欠損補完前）:input_train_file', X.isnull().sum().sum())
    print(X.isnull().sum())
    return X, y

In [ ]:
def input_test_file(filename, my_dtype, id_label):
    df_s = pd.read_csv(filename, header=0, dtype=my_dtype)
    #df_s = df_s[df_s['room_type'] == 'Shared room']
    #df_s = df_s[df_s['room_type'] == 'Entire home/apt']
    #df_s = df_s[df_s['room_type'] == 'Private room']
    df_s = df_s.reset_index(drop=True)
    ID_s = df_s.iloc[:,[0]]
    X_s  = df_s.drop(id_label, axis=1)
    return df_s, X_s

In [ ]:
import pandas as pd
import datetime as dt
import re
def transform_data(X:pd.core.series.Series):
    X['log_beds'] = np.log1p(X['beds'])
    X['log_bathrooms'] = np.log1p(X['bathrooms'])
    X['log_bedrooms'] = np.log1p(X['bedrooms'])
    X['log_number_of_reviews'] = np.log1p(X['number_of_reviews'])
    X['log_review_scores_rating'] = np.log1p(X['review_scores_rating'])
    #X = X.drop('beds', axis=1)
    #X = X.drop('bathrooms', axis=1)
    #X = X.drop('bedrooms', axis=1)
    #X = X.drop('number_of_reviews', axis=1)
    #X = X.drop('review_scores_rating', axis=1)
    def change_tf(x):
        if x == 't':
            return 1
        elif x == 'f':
            return 0
    X['cleaning_fee'] = X['cleaning_fee'].apply(change_tf)
    X['host_identity_verified'] = X['host_identity_verified'].apply(change_tf)
    X['instant_bookable'] = X['instant_bookable'].apply(change_tf)
    X['host_since'] =  pd.to_datetime(X['host_since']).map(pd.Timestamp.to_julian_date)
    X['first_review'] =  pd.to_datetime(X['first_review']).map(pd.Timestamp.to_julian_date)
    X['last_review'] =  pd.to_datetime(X['last_review']).map(pd.Timestamp.to_julian_date)
    X['host_response_rate'] = X['host_response_rate'].str.strip('%')
    X["length_description"] = X["description"].apply(lambda x: len(x))
    X["log_length_description"] = np.log1p(X["length_description"])
    X["length_name"] = X["name"].apply(lambda x: len(x))
    X["amenities"] = X["amenities"].str.replace('{', '')
    X["amenities"] = X["amenities"].str.replace('}', '')
    X["amenities"] = X["amenities"].str.replace('"', '')
    X["amenities"] = X["amenities"].str.replace("'", '')
    X["amenities"] = X["amenities"].str.replace("’", '')
    X["amenities"] = X["amenities"].str.strip("'")
    X["amenities"] = X["amenities"].str.strip("[")
    X["amenities"] = X["amenities"].str.strip("]")
    X["amenities"] = X["amenities"].str.strip("’")
    X["count_amenities"] = X["amenities"].astype(str).apply(lambda x: len(x.split(",")))
    X["log_count_amenities"] = np.log1p(X["count_amenities"])
    X['thumbnail_url'] = X['thumbnail_url'].astype(str)
    X['thumbnail_url'] = X['thumbnail_url'].fillna('')
    X["has_url"] = X["thumbnail_url"].apply(lambda x : 1 if len(x) == 0 else 0)
    #X["length_url"] = X["thumbnail_url"].apply(lambda x: len(x))
    #X["log_length_url"] = np.log1p(X["length_url"])
    X['zipcode'] = X['zipcode'].replace("11385-2308", 11385-2308)
    X['zipcode'] = X['zipcode'].replace("91606-1412", 91606-1412)
    X['zipcode'] = X['zipcode'].replace("91606-1412", 91606-1412)
    X['zipcode'] = X['zipcode'].replace("1m", 1000)
    X['zipcode'] = X['zipcode'].replace("Near 91304", 91304)
    X['zipcode'] = X['zipcode'].replace("11249\r\r\r\r\r\r\n11249", 11249)
    X['zipcode'] = X['zipcode'].replace("11426-1175", 11426-1175)
    X['zipcode'] = X['zipcode'].replace("90039-2715", 90039-2715)
    X['zipcode'] = X['zipcode'].replace("90403-2638", 90403-2638)
    X['zipcode'] = X['zipcode'].replace("60660-1448", 60660-1448)
    X['zipcode'] = X['zipcode'].replace("90036-2514", 90036-2514)
    X['zipcode'] = X['zipcode'].replace("11103-3233", 11103-3233)
    X['zipcode'] = X['zipcode'].replace('10002-2289', 10002-2289)
    X['zipcode'] = X['zipcode'].replace('20007-3413', 20007-3413)
    X['zipcode'] = X['zipcode'].replace('91001-2243', 91001-2243)
    X['zipcode'] = X['zipcode'].replace('20001-4044', 20001-4044)
    X['zipcode'] = X['zipcode'].replace('10003-8623', 10003-8623)
    X['zipcode'] = X['zipcode'].replace('95202\r\r\r\r\r\r\n\r\r\r\r\r\r\n\r\r\r\r\r\r\n94158', 94158)
    X['zipcode'] = X['zipcode'].replace('90035-4475', 90035-4475)
    X['zipcode'] = X['zipcode'].replace('90034-2203', 90034-2203)
    X['zipcode'] = X['zipcode'].replace(' ', 0)
    X['zipcode'] = X['zipcode'].fillna(0)
    X['zipcode'] = X['zipcode'].astype(str)
    X['zipcode'] = X['zipcode'].astype(float)
    X['zipcode'] = X['zipcode'].round().astype(int)
    #X['log_zipcode'] = np.log1p(X['zipcode'])
    #X['log_latitude'] = np.log1p(X['latitude'])
    #X['log_longitude'] = np.log1p(X['longitude'])
    df_amenities = X["amenities"].astype(str).str.split(',', expand=False)
    nd = df_amenities.values.tolist()
    import itertools
    nd = list(itertools.chain.from_iterable(nd))
    def contain_item(item, word):
        if word in item:
            return 1
        return 0
    for word in np.unique(nd):
        if word != '':      
            X["amenities_" + word] = X["amenities"].apply(contain_item, args=(word,)).astype(str)
    # 
    X['frequency_encode_city'] = function.convert_to_frequency_encode(X, 'city')
    X['frequency_encode_amenities'] = function.convert_to_frequency_encode(X, 'amenities')
    X['frequency_encode_zipcode'] = function.convert_to_frequency_encode(X, 'zipcode')
    X['frequency_encode_bed_type'] = function.convert_to_frequency_encode(X, 'bed_type')
    X['frequency_encode_room_type'] = function.convert_to_frequency_encode(X, 'room_type')
    X['frequency_encode_neighbourhood'] = function.convert_to_frequency_encode(X, 'neighbourhood')
    X['frequency_encode_cancellation_policy'] = function.convert_to_frequency_encode(X, 'cancellation_policy')

    X['label_encode_city'] = function.convert_to_label_encode(X, 'city')
    X['label_encode_amenities'] = function.convert_to_label_encode(X, 'amenities')
    X['label_encode_zipcode'] = function.convert_to_label_encode(X, 'zipcode')
    X['label_encode_bed_type'] = function.convert_to_label_encode(X, 'bed_type')
    X['label_encode_room_type'] = function.convert_to_label_encode(X, 'room_type')
    X['label_encode_neighbourhood'] = function.convert_to_label_encode(X, 'neighbourhood')
    X['label_encode_cancellation_policy'] = function.convert_to_label_encode(X, 'cancellation_policy')

    X['count_encode_city'] = function.convert_to_count_encode(X, 'city')
    X['count_encode_amenities'] = function.convert_to_count_encode(X, 'amenities')
    X['count_encode_zipcode'] = function.convert_to_count_encode(X, 'zipcode')
    X['count_encode_bed_type'] = function.convert_to_count_encode(X, 'bed_type')
    X['count_encode_room_type'] = function.convert_to_count_encode(X, 'room_type')
    X['count_encode_neighbourhood'] = function.convert_to_count_encode(X, 'neighbourhood')
    X['count_encode_cancellation_policy'] = function.convert_to_count_encode(X, 'cancellation_policy')

    X['label_count_encode_city'] = function.convert_to_label_count_encode(X, 'city')
    X['label_count_encode_amenities'] = function.convert_to_label_count_encode(X, 'amenities')
    X['label_count_encode_zipcode'] = function.convert_to_label_count_encode(X, 'zipcode')
    X['label_count_encode_bed_type'] = function.convert_to_label_count_encode(X, 'bed_type')
    X['label_count_encode_room_type'] = function.convert_to_label_count_encode(X, 'room_type')
    X['label_count_encode_neighbourhood'] = function.convert_to_label_count_encode(X, 'neighbourhood')
    X['label_count_encode_cancellation_policy'] = function.convert_to_label_count_encode(X, 'cancellation_policy')

    #X['target_encode_amenities'] = convert_to_target_encode(X, 'amenities')
    #X['target_encode_zipcode'] = convert_to_target_encode(X, 'zipcode')
    #X['target_encode_bed_type'] = convert_to_target_encode(X, 'bed_type')
    #X['target_encode_room_type'] = convert_to_target_encode(X, 'room_type')
    #X['target_encode_neighbourhood'] = convert_to_target_encode(X, 'neighbourhood')
    #X['target_encode_cancellation_policy'] = convert_to_target_encode(X, 'cancellation_policy')
    #X['label_count_encode_city'] = convert_to_label_encode(X, 'amenities')
    #X['label_count_encode_city'] = convert_to_label_encode(X, 'amenities')
    #X['label_count_encode_city'] = convert_to_label_encode(X, 'amenities')

    # 不要カラム削除
    #X = X.drop('zipcode', axis=1)
    #X = X.drop('city', axis=1)
    #X = X.drop('amenities', axis=1)
    for column in drop_columns:
        X = X.drop(column, axis=1)
    X = X.reset_index(drop=True)

    return X

In [ ]:
input_evaluation = function.choice(options_evaluation)

In [ ]:
X, y = input_train_file(train_file_name, my_dtype, id_label)

Preprocessing

In [ ]:
X = transform_data(X)

In [ ]:
X_ohe = function.one_hot_encoding(X, ohe_columns)
print('欠損個数（数値変数の欠損補完前）', X_ohe.isnull().sum().sum())
print('')
print(X_ohe.isnull().sum())
X_ohe, X_ohe_columns = function.imputation(out_put_data_dir, model_columns_file_name, X_ohe)

Train

In [ ]:
import time
start_at = time.time()
X_train, X_valid, y_train, y_valid = function.train_model(out_put_data_dir, pipelines, feature_selection_rf_list, n_features_to_select, X_ohe, y, X_ohe_columns, input_evaluation.value, is_holdout, is_optuna)
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

In [ ]:
function_evaluation = function.decide_evaluation(input_evaluation)

In [ ]:
function.display_evaluation(is_holdout, out_put_data_dir, pipelines, function_evaluation, input_evaluation, X_train, y_train, X_valid, y_valid)

In [ ]:
input_algorithm = function.choice(function.options_algorithm)

In [ ]:
if is_k_fold:
    from sklearn.metrics import make_scorer
    scorer = make_scorer(function.root_mean_squared_error, greater_is_better=False)
    function.cross_validatior(scorer, out_put_data_dir, n_features_to_select, pipelines, input_evaluation, X_ohe, y)

In [ ]:
def main():
    algorithm_name = input_algorithm.value
    df_s, X_s = input_test_file(test_file_name, my_dtype, id_label)
    X_s = transform_data(X_s)
    X_ohe_s = function.one_hot_encoding(X_s, ohe_columns)
    X_predicted = function.preprocessing(out_put_data_dir, model_columns_file_name, algorithm_name, X_ohe, X_ohe_s)
    predict = function.scoring(out_put_data_dir, algorithm_name, X_predicted);
    predict = np.expm1(predict)
    if is_target_minus_to_zero:
        predict = [i if i > 0 else 0 for i in predict]
    function.output_file(out_put_data_dir, n_features_to_select, target_label, df_s, id_label, predict, algorithm_name, file_extention, header=False)
    print(input_evaluation.value + ' selected')
    print(algorithm_name + ' selected')
    if algorithm_name in function.feature_importances_algorithm_list:
        pd.options.display.max_columns = 100
        pd.set_option('display.max_rows', 100)
        feature_importances = pipelines[algorithm_name]['est'].feature_importances_
        feature_importances = pd.Series(feature_importances, index=X_predicted.columns.values.tolist())
        sorted_feature_importances = sorted(feature_importances.items(), key=lambda x:-x[1])
        #print(sorted_feature_importances)
        str_all_print = 'n_features_to_select,' + str(n_features_to_select) + '\n'
        str_all_print += 'Evaluation index' + ',' + input_evaluation.value + '\n'
        print('評価指標:' + input_evaluation.value)
        for index, value in sorted_feature_importances:
            str_print = str(index) + ',' + str(value) + '\n'
            str_all_print += str_print
        import datetime
        with open(out_put_data_dir + 'feature_importances' + '_' + str(n_features_to_select) + "_" +  datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.csv', mode='w') as f:
            f.write(str_all_print)
        display(pd.DataFrame(sorted_feature_importances))

In [ ]:
if __name__ == '__main__':
    main()